# Setup

In [23]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, inspect, text
import os

#Data Viz
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interact

#Estatística
from scipy.stats import normaltest
from scipy.stats import norm
from statsmodels.stats.weightstats import DescrStatsW

In [24]:
engine = create_engine(os.environ["REDSHIFT_READ_URI"], echo=False)

def sql_df(query):
  with engine.connect() as conexao:
    consulta = conexao.execute(text(query))
    dados = consulta.fetchall()
  df = pd.DataFrame(dados, columns= consulta.keys())
  return df

# TESTE L1

## Importação + Tratamento dos dados

In [25]:
query = '''

select
	id,
	empresa_id,
	regua_id,
	evento_regua_id,
	dia,
	tipo,
	data_ref,
	enviado_em::date,
	criado_em::date,
	status,
	cliente_id,
	contrato_id,
	pagamento_id,
	url_evidencia,
	parcela_id,
	detalhes,
	motivo_nao_enviado,
	motivo_erro,
	remetente_dominio,
	tipo_notificacao,
	fluxo_automatico_id,
	contato_id,
	mes_enviado_em,
	ids_multiplos_contratos,
	ultima_interacao,
	nps_id,
	pesquisa_id,
	movimentacao_id,
	whatsapp_iniciativa_id,
	broker
from
	melembre_lake.notificacao n
where
	enviado_em::date between '2024-05-19' and '2024-07-20'
	and regua_id in (516, 517, 523, 524, 525, 526, 521, 522)
    and dia = -1
    and tipo in ('SMS','WHATSAPP')
    and status != 'NAO_ENVIADO'


'''

In [26]:
base = sql_df(query)

In [27]:
regua_516 = base[base['regua_id'] == 516].groupby('enviado_em').size().reset_index(name='Envios')
regua_517 = base[base['regua_id'] == 517].groupby('enviado_em').size().reset_index(name='Envios')
regua_523 = base[base['regua_id'] == 523].groupby('enviado_em').size().reset_index(name='Envios')
regua_524 = base[base['regua_id'] == 524].groupby('enviado_em').size().reset_index(name='Envios')
regua_525 = base[base['regua_id'] == 525].groupby('enviado_em').size().reset_index(name='Envios')
regua_526 = base[base['regua_id'] == 526].groupby('enviado_em').size().reset_index(name='Envios')
regua_521 = base[base['regua_id'] == 521].groupby('enviado_em').size().reset_index(name='Envios')
regua_522 = base[base['regua_id'] == 522].groupby('enviado_em').size().reset_index(name='Envios')

In [28]:
interacao_516 = base[(base['regua_id'] == 516) & (base['ultima_interacao'].notnull())].groupby('enviado_em').size().reset_index(name='Envios')
interacao_517 = base[(base['regua_id'] == 517) & (base['ultima_interacao'].notnull())].groupby('enviado_em').size().reset_index(name='Envios')
interacao_523 = base[(base['regua_id'] == 523) & (base['ultima_interacao'].notnull())].groupby('enviado_em').size().reset_index(name='Envios')
interacao_524 = base[(base['regua_id'] == 524) & (base['ultima_interacao'].notnull())].groupby('enviado_em').size().reset_index(name='Envios')
interacao_525 = base[(base['regua_id'] == 525) & (base['ultima_interacao'].notnull())].groupby('enviado_em').size().reset_index(name='Envios')
interacao_526 = base[(base['regua_id'] == 526) & (base['ultima_interacao'].notnull())].groupby('enviado_em').size().reset_index(name='Envios')
interacao_521 = base[(base['regua_id'] == 521) & (base['ultima_interacao'].notnull())].groupby('enviado_em').size().reset_index(name='Envios')
interacao_522 = base[(base['regua_id'] == 522) & (base['ultima_interacao'].notnull())].groupby('enviado_em').size().reset_index(name='Envios')


In [29]:
base_516 = regua_516.merge(interacao_516, left_on='enviado_em', right_on='enviado_em', how='left',suffixes=('_regua', '_interacao')).fillna(0)
base_517 = regua_517.merge(interacao_517, left_on='enviado_em', right_on='enviado_em', how='left',suffixes=('_regua', '_interacao')).fillna(0)
base_523  = regua_523.merge(interacao_523, left_on='enviado_em', right_on='enviado_em', how='left',suffixes=('_regua', '_interacao')).fillna(0)
base_524 = regua_524.merge(interacao_524, left_on='enviado_em', right_on='enviado_em', how='left',suffixes=('_regua', '_interacao')).fillna(0)
base_525 = regua_525.merge(interacao_525, left_on='enviado_em', right_on='enviado_em', how='left',suffixes=('_regua', '_interacao')).fillna(0)
base_526 = regua_526.merge(interacao_526, left_on='enviado_em', right_on='enviado_em', how='left',suffixes=('_regua', '_interacao')).fillna(0)
base_521 = regua_521.merge(interacao_521, left_on='enviado_em', right_on='enviado_em', how='left',suffixes=('_regua', '_interacao')).fillna(0)
base_522 = regua_522.merge(interacao_522, left_on='enviado_em', right_on='enviado_em', how='left',suffixes=('_regua', '_interacao')).fillna(0)

In [30]:
base_516['Tx_Interesse'] = round((base_516['Envios_interacao'] / base_516['Envios_regua'] ) * 100, 2)
base_517['Tx_Interesse'] = round((base_517['Envios_interacao'] / base_517['Envios_regua'] ) * 100, 2)
base_523['Tx_Interesse'] = round(( base_523['Envios_interacao'] / base_523['Envios_regua'] ) * 100,2)
base_524['Tx_Interesse'] = round(( base_524['Envios_interacao'] / base_524['Envios_regua']) * 100, 2)
base_525['Tx_Interesse'] = round((base_525['Envios_interacao'] / base_525['Envios_regua']) * 100,2)
base_526['Tx_Interesse'] = round((base_526['Envios_interacao'] / base_526['Envios_regua']) * 100,2)
base_521['Tx_Interesse'] = round((base_521['Envios_interacao'] / base_521['Envios_regua']) * 100,2)
base_522['Tx_Interesse'] = round((base_522['Envios_interacao'] / base_522['Envios_regua']) * 100,2)

## Análise Gráfica + Teste de Hipóteses

### Régua 516 x Régua 517

In [31]:
tx_geral_1 = pd.DataFrame({'Regua':['517 (SMS)','516 (Whatsapp)'],
                         'Taxa':[round((sum(base_517["Envios_interacao"]) / sum(base_517["Envios_regua"])) * 100,2),
                                 round((sum(base_516["Envios_interacao"]) / sum(base_516["Envios_regua"])) * 100,2)]})

In [32]:
fig = make_subplots(rows=3,cols=1,shared_yaxes= True, subplot_titles=('Taxa Interesse Régua 517 (SMS)','Taxa Interesse Régua 516 (Whatsapp)','Taxa Geral'))
fig.add_trace(go.Bar(x=base_517["enviado_em"],
                     y=base_517["Tx_Interesse"],
                     marker_color="red",
                     text=base_517["Tx_Interesse"]),row=1,col=1)
fig.add_trace(go.Bar(x=base_516["enviado_em"],
                     y=base_516["Tx_Interesse"],
                     marker_color="blue",
                     text=base_516["Tx_Interesse"]),row=2,col=1)
fig.add_trace(go.Bar(x=tx_geral_1['Regua'],
                     y=tx_geral_1['Taxa'],
                     marker_color="Green",
                     text=tx_geral_1['Taxa']),row=3,col=1)
fig.update_yaxes(range=[0, 100], row=1,col=1)
fig.update_yaxes(range=[0, 100], row=2,col=1)
fig.update_yaxes(range=[0, 100], row=3,col=1)
fig.update_layout(width=1400, height=1250, showlegend= False)

Conforme análise gráfica, a Régua A (SMS) desempenhou melhor que a Régua B, seguirei com alguns testes de hipóteses para validar a questão.

Existe diferença significativa na quantidade disparada nas réguas?

H0: Não existe Diferença


H1: Existe diferença
***

In [33]:
total_envios_1 = pd.DataFrame({'Reguas':['517 (SMS)','516 (Whatsapp)'],
                             'Total Notificações':[base_517['Envios_regua'].sum(),base_516['Envios_regua'].sum()]})

fig = go.Figure()
fig.add_trace(go.Bar(x=total_envios_1['Reguas'],
                     y=total_envios_1['Total Notificações'],
                     marker_color="red",
                     text=total_envios_1['Total Notificações']))
fig.update_layout(title='Total Disparos')

Conforme validações da análise gráfica foi possível ver que houve mais envios na régua referente ao SMS, e isso pode ter impactado o resultado final. Por isso será para o teste de hipótese da Regua Desafiante (516) ter tido um melhor desempenho em relação a Regua Controle (517), será utilizado um teste de duas proporções a fim de manter as proporções de disparos e interesse. 

H0 - A taxa de interesse da Regua 516 é menor que a regua 517

H1 - A taxa de interesse da Regua 516 é maior que a regua 517

In [34]:
pa = sum(base_516['Envios_interacao']) / sum(base_516['Envios_regua'])  # grupo desafiante (516)
pb = sum(base_517['Envios_interacao']) / sum(base_517['Envios_regua'])  # grupo controle (517)
pc = (sum(base_516['Envios_interacao']) + sum(base_517['Envios_interacao'])) / (sum(base_516['Envios_regua']) + sum(base_517['Envios_regua']))

SE = np.sqrt((pc * (1-pc)) * ((1 / sum(base_516['Envios_regua'])) + (1 / sum(base_517['Envios_regua']))))
Z = (pa - pb) / SE

p_valor = (norm.sf(Z))

if p_valor <= 0.05:
    print('Aceitar H1')
else:
    print('Aceitar H0')

Aceitar H1


Segundo o teste de hipóteses não existe evidências estatísticas suficientes para acatar que a régua desafiante (516) desempenhou melhor que a régua controle (517).

***

### Régua 521 x Régua 522

In [35]:
tx_geral_2 = pd.DataFrame({'Regua':['522 (SMS)','521 (Whatsapp)'],
                         'Taxa':[round((sum(base_522["Envios_interacao"]) / sum(base_522["Envios_regua"])) * 100,2),
                                 round((sum(base_521["Envios_interacao"]) / sum(base_521["Envios_regua"])) * 100,2)]})

In [36]:
fig = make_subplots(rows=3,cols=1,shared_yaxes= True, subplot_titles=('Taxa Interesse Régua 522 (SMS)','Taxa Interesse Régua 521 (Whatsapp)','Taxa Geral'))
fig.add_trace(go.Bar(x=base_522["enviado_em"],
                     y=base_522["Tx_Interesse"],
                     marker_color="red",
                     text=base_522["Tx_Interesse"]),row=1,col=1)
fig.add_trace(go.Bar(x=base_521["enviado_em"],
                     y=base_521["Tx_Interesse"],
                     marker_color="blue",
                     text=base_521["Tx_Interesse"]),row=2,col=1)
fig.add_trace(go.Bar(x=tx_geral_2['Regua'],
                     y=tx_geral_2['Taxa'],
                     marker_color="Green",
                     text=tx_geral_2['Taxa']),row=3,col=1)
fig.update_yaxes(range=[0, 100], row=1,col=1)
fig.update_yaxes(range=[0, 100], row=2,col=1)
fig.update_yaxes(range=[0, 100], row=3,col=1)
fig.update_layout(width=1400, height=1250, showlegend= False)

Conforme análise gráfica, a Régua A (SMS) desempenhou melhor que a Régua B, seguirei com alguns testes de hipóteses para validar a questão.

Existe diferença significativa na quantidade disparada nas réguas?

H0: Não existe Diferença


H1: Existe diferença
***

In [37]:
total_envios_2 = pd.DataFrame({'Reguas':['522 (SMS)','521 (Whatsapp)'],
                             'Total Notificações':[base_522['Envios_regua'].sum(),base_521['Envios_regua'].sum()]})

fig = go.Figure()
fig.add_trace(go.Bar(x=total_envios_2['Reguas'],
                     y=total_envios_2['Total Notificações'],
                     marker_color="red",
                     text=total_envios_2['Total Notificações']))
fig.update_layout(title='Total Disparos')

Conforme validações da análise gráfica foi possível ver que houve mais envios na régua referente ao SMS, e isso pode ter impactado o resultado final. Por isso será para o teste de hipótese da Regua Desafiante (521) ter tido um melhor desempenho em relação a Regua Controle (522), será utilizado um teste de duas proporções a fim de manter as proporções de disparos e interesse. 

H0 - A taxa de interesse da Regua 521 é menor que a regua 522

H1 - A taxa de interesse da Regua 521 é maior que a regua 522

In [38]:
pa = sum(base_521['Envios_interacao']) / sum(base_521['Envios_regua'])  # grupo desafiante (521)
pb = sum(base_522['Envios_interacao']) / sum(base_522['Envios_regua'])  # grupo controle (522)
pc = (sum(base_521['Envios_interacao']) + sum(base_522['Envios_interacao'])) / (sum(base_521['Envios_regua']) + sum(base_522['Envios_regua']))

SE = np.sqrt((pc * (1-pc)) * ((1 / sum(base_521['Envios_regua'])) + (1 / sum(base_522['Envios_regua']))))
Z = (pa - pb) / SE

p_valor = (norm.sf(Z))

if p_valor <= 0.05:
    print('Aceitar H1')
else:
    print('Aceitar H0')

Aceitar H0


Segundo o teste de hipóteses não existe evidências estatísticas suficientes para acatar que a régua desafiante (521) desempenhou melhor que a régua controle (522).

***

### Régua 523 x Régua 524

In [39]:
tx_geral_3 = pd.DataFrame({'Regua':['524 (SMS)','523 (Whatsapp)'],
                         'Taxa':[round((sum(base_524["Envios_interacao"]) / sum(base_524["Envios_regua"])) * 100,2),
                                 round((sum(base_523["Envios_interacao"]) / sum(base_523["Envios_regua"])) * 100,2)]})

In [40]:
fig = make_subplots(rows=3,cols=1,shared_yaxes= True, subplot_titles=('Taxa Interesse Régua 524 (SMS)','Taxa Interesse Régua 523 (Whatsapp)','Taxa Geral'))

fig.add_trace(go.Bar(x=base_524["enviado_em"],
                     y=base_524["Tx_Interesse"],
                     marker_color="red",
                     text=base_524["Tx_Interesse"]),row=1,col=1)
fig.add_trace(go.Bar(x=base_523["enviado_em"],
                     y=base_523["Tx_Interesse"],
                     marker_color="blue",
                     text=base_523["Tx_Interesse"]),row=2,col=1)
fig.add_trace(go.Bar(x=tx_geral_3['Regua'],
                     y=tx_geral_3['Taxa'],
                     marker_color="Green",
                     text=tx_geral_3['Taxa']),row=3,col=1)
fig.update_yaxes(range=[0, 100], row=1,col=1)
fig.update_yaxes(range=[0, 100], row=2,col=1)
fig.update_yaxes(range=[0, 100], row=3,col=1)
fig.update_layout(width=1400, height=1250, showlegend= False)

Conforme análise gráfica, a Régua A (SMS) desempenhou melhor que a Régua B, seguirei com alguns testes de hipóteses para validar a questão.

Existe diferença significativa na quantidade disparada nas réguas?

H0: Não existe Diferença


H1: Existe diferença
***

In [41]:
total_envios_3 = pd.DataFrame({'Reguas':['524 (SMS)','523 (Whatsapp)'],
                             'Total Notificações':[base_524['Envios_regua'].sum(),base_523['Envios_regua'].sum()]})

fig = go.Figure()
fig.add_trace(go.Bar(x=total_envios_3['Reguas'],
                     y=total_envios_3['Total Notificações'],
                     marker_color="red",
                     text=total_envios_3['Total Notificações']))
fig.update_layout(title='Total Disparos')

Conforme validações da análise gráfica foi possível ver que houve mais envios na régua referente ao SMS, e isso pode ter impactado o resultado final. Por isso será para o teste de hipótese da Regua Desafiante (523) ter tido um melhor desempenho em relação a Regua Controle (524), será utilizado um teste de duas proporções a fim de manter as proporções de disparos e interesse. 

H0 - A taxa de interesse da Regua 523 é menor que a regua 524

H1 - A taxa de interesse da Regua 523 é maior que a regua 524

In [42]:
pa = sum(base_523['Envios_interacao']) / sum(base_523['Envios_regua'])  # grupo desafiante (523)
pb = sum(base_524['Envios_interacao']) / sum(base_524['Envios_regua'])  # grupo controle (524)
pc = (sum(base_523['Envios_interacao']) + sum(base_524['Envios_interacao'])) / (sum(base_523['Envios_regua']) + sum(base_524['Envios_regua']))

SE = np.sqrt((pc * (1-pc)) * ((1 / sum(base_523['Envios_regua'])) + (1 / sum(base_524['Envios_regua']))))
Z = (pa - pb) / SE

p_valor = (norm.sf(Z))

if p_valor <= 0.05:
    print('Aceitar H1')
else:
    print('Aceitar H0')

Aceitar H1


Segundo o teste de hipóteses não existe evidências estatísticas suficientes para acatar que a régua desafiante (523) desempenhou melhor que a régua controle (524).

***

### Régua 525 x Régua 526

In [43]:
tx_geral_4 = pd.DataFrame({'Regua':['526 (SMS)','525 (Whatsapp)'],
                         'Taxa':[round((sum(base_526["Envios_interacao"]) / sum(base_526["Envios_regua"])) * 100,2),
                                 round((sum(base_525["Envios_interacao"]) / sum(base_525["Envios_regua"])) * 100,2)]})

In [44]:
fig = make_subplots(rows=3,cols=1,shared_yaxes= True, subplot_titles=('Taxa Interesse Régua 526 (SMS)','Taxa Interesse Régua 525 (Whatsapp)','Taxa Geral'))
fig.add_trace(go.Bar(x=base_526["enviado_em"],
                     y=base_526["Tx_Interesse"],
                     marker_color="red",
                     text=base_526["Tx_Interesse"]),row=1,col=1)
fig.add_trace(go.Bar(x=base_525["enviado_em"],
                     y=base_525["Tx_Interesse"],
                     marker_color="blue",
                     text=base_525["Tx_Interesse"]),row=2,col=1)
fig.add_trace(go.Bar(x=tx_geral_4['Regua'],
                     y=tx_geral_4['Taxa'],
                     marker_color="Green",
                     text=tx_geral_4['Taxa']),row=3,col=1)
fig.update_yaxes(range=[0, 100], row=1,col=1)
fig.update_yaxes(range=[0, 100], row=2,col=1)
fig.update_yaxes(range=[0, 100], row=3,col=1)
fig.update_layout(width=1400, height=1250, showlegend= False)

Conforme análise gráfica, a Régua A (SMS) desempenhou melhor que a Régua B, seguirei com alguns testes de hipóteses para validar a questão.

Existe diferença significativa na quantidade disparada nas réguas?

H0: Não existe Diferença


H1: Existe diferença
***

In [45]:
total_envios_4 = pd.DataFrame({'Reguas':['526 (SMS)','525 (Whatsapp)'],
                             'Total Notificações':[base_526['Envios_regua'].sum(),base_525['Envios_regua'].sum()]})

fig = go.Figure()
fig.add_trace(go.Bar(x=total_envios_4['Reguas'],
                     y=total_envios_4['Total Notificações'],
                     marker_color="red",
                     text=total_envios_4['Total Notificações']))
fig.update_layout(title='Total Disparos')

Conforme validações da análise gráfica foi possível ver que houve mais envios na régua referente ao SMS, e isso pode ter impactado o resultado final. Por isso será para o teste de hipótese da Regua Desafiante (523) ter tido um melhor desempenho em relação a Regua Controle (524), será utilizado um teste de duas proporções a fim de manter as proporções de disparos e interesse. 

H0 - A taxa de interesse da Regua 523 é menor que a regua 524

H1 - A taxa de interesse da Regua 523 é maior que a regua 524

In [46]:
pa = sum(base_525['Envios_interacao']) / sum(base_525['Envios_regua'])  # grupo desafiante (525)
pb = sum(base_526['Envios_interacao']) / sum(base_526['Envios_regua'])  # grupo controle (526)
pc = (sum(base_525['Envios_interacao']) + sum(base_526['Envios_interacao'])) / (sum(base_525['Envios_regua']) + sum(base_526['Envios_regua']))

SE = np.sqrt((pc * (1-pc)) * ((1 / sum(base_525['Envios_regua'])) + (1 / sum(base_526['Envios_regua']))))
Z = (pa - pb) / SE

p_valor = (norm.sf(Z))

if p_valor <= 0.05:
    print('Aceitar H1')
else:
    print('Aceitar H0')

Aceitar H0


Segundo o teste de hipóteses não existe evidências estatísticas suficientes para acatar que a régua desafiante (525) desempenhou melhor que a régua controle (526).

***

# TESTE A1

## Importação + Tratamento dos dados

In [47]:
query = '''

select
	id,
	empresa_id,
	regua_id,
	evento_regua_id,
	dia,
	tipo,
	data_ref,
	enviado_em::date,
	criado_em::date,
	status,
	cliente_id,
	contrato_id,
	pagamento_id,
	url_evidencia,
	parcela_id,
	detalhes,
	motivo_nao_enviado,
	motivo_erro,
	remetente_dominio,
	tipo_notificacao,
	fluxo_automatico_id,
	contato_id,
	mes_enviado_em,
	ids_multiplos_contratos,
	ultima_interacao,
	nps_id,
	pesquisa_id,
	movimentacao_id,
	whatsapp_iniciativa_id,
	broker
from
	melembre_lake.notificacao n
where
	enviado_em::date between '2024-07-24' and '2024-08-09'
	and regua_id in (536, 537, 538, 539)
    and dia = 1
    and tipo in ('SMS','WHATSAPP')
    and status != 'NAO_ENVIADO'


'''

In [48]:
base = sql_df(query)

In [76]:
regua_536 = base[base['regua_id'] == 536].groupby('enviado_em').size().reset_index(name='Envios')
regua_537 = base[base['regua_id'] == 537].groupby('enviado_em').size().reset_index(name='Envios')
regua_538 = base[base['regua_id'] == 538].groupby('enviado_em').size().reset_index(name='Envios')
regua_539 = base[base['regua_id'] == 539].groupby('enviado_em').size().reset_index(name='Envios')

In [85]:
interacao_536 = base[(base['regua_id'] == 536) & (base['ultima_interacao'].notnull())].groupby('enviado_em').size().reset_index(name='Envios')
interacao_537 = base[(base['regua_id'] == 537) & (base['ultima_interacao'].notnull())].groupby('enviado_em').size().reset_index(name='Envios')
interacao_538 = base[(base['regua_id'] == 538) & (base['ultima_interacao'].notnull())].groupby('enviado_em').size().reset_index(name='Envios')
interacao_539 = base[(base['regua_id'] == 539) & (base['ultima_interacao'].notnull())].groupby('enviado_em').size().reset_index(name='Envios')

In [86]:
base_536 = regua_536.merge(interacao_536, left_on='enviado_em', right_on='enviado_em', how='left',suffixes=('_regua', '_interacao')).fillna(0)
base_537 = regua_537.merge(interacao_537, left_on='enviado_em', right_on='enviado_em', how='left',suffixes=('_regua', '_interacao')).fillna(0)
base_538  = regua_538.merge(interacao_538, left_on='enviado_em', right_on='enviado_em', how='left',suffixes=('_regua', '_interacao')).fillna(0)
base_539 = regua_539.merge(interacao_539, left_on='enviado_em', right_on='enviado_em', how='left',suffixes=('_regua', '_interacao')).fillna(0)

In [87]:
base_536['Tx_Interesse'] = round((base_536['Envios_interacao'] / base_536['Envios_regua'] ) * 100, 2)
base_537['Tx_Interesse'] = round((base_537['Envios_interacao'] / base_537['Envios_regua'] ) * 100, 2)
base_538['Tx_Interesse'] = round(( base_538['Envios_interacao'] / base_538['Envios_regua'] ) * 100,2)
base_539['Tx_Interesse'] = round(( base_539['Envios_interacao'] / base_539['Envios_regua']) * 100, 2)

## Análise Gráfica + Teste de Hipóteses

### Régua 536 x Régua 537

In [88]:
tx_geral_1 = pd.DataFrame({'Regua':['537 (SMS)','536 (Whatsapp)'],
                         'Taxa':[round((sum(base_537["Envios_interacao"]) / sum(base_537["Envios_regua"])) * 100,2),
                                 round((sum(base_536["Envios_interacao"]) / sum(base_536["Envios_regua"])) * 100,2)]})

In [89]:
fig = make_subplots(rows=3,cols=1,shared_yaxes= True, subplot_titles=('Taxa Interesse Régua 537 (SMS)','Taxa Interesse Régua 536 (Whatsapp)','Taxa Geral'))
fig.add_trace(go.Bar(x=base_537["enviado_em"],
                     y=base_537["Tx_Interesse"],
                     marker_color="red",
                     text=base_537["Tx_Interesse"]),row=1,col=1)
fig.add_trace(go.Bar(x=base_536["enviado_em"],
                     y=base_536["Tx_Interesse"],
                     marker_color="blue",
                     text=base_536["Tx_Interesse"]),row=2,col=1)
fig.add_trace(go.Bar(x=tx_geral_1['Regua'],
                     y=tx_geral_1['Taxa'],
                     marker_color="Green",
                     text=tx_geral_1['Taxa']),row=3,col=1)
fig.update_yaxes(range=[0, 100], row=1,col=1)
fig.update_yaxes(range=[0, 100], row=2,col=1)
fig.update_yaxes(range=[0, 100], row=3,col=1)
fig.update_layout(width=1400, height=1250, showlegend= False)

Conforme análise gráfica, a Régua A (SMS) desempenhou melhor que a Régua B, seguirei com alguns testes de hipóteses para validar a questão.

Existe diferença significativa na quantidade disparada nas réguas?

H0: Não existe Diferença


H1: Existe diferença
***

In [90]:
total_envios_1 = pd.DataFrame({'Reguas':['537 (SMS)','536 (Whatsapp)'],
                             'Total Notificações':[base_537['Envios_regua'].sum(),base_536['Envios_regua'].sum()]})

fig = go.Figure()
fig.add_trace(go.Bar(x=total_envios_1['Reguas'],
                     y=total_envios_1['Total Notificações'],
                     marker_color="red",
                     text=total_envios_1['Total Notificações']))
fig.update_layout(title='Total Disparos')

Conforme validações da análise gráfica foi possível ver que houve mais envios na régua referente ao SMS, e isso pode ter impactado o resultado final. Por isso será para o teste de hipótese da Regua Desafiante (516) ter tido um melhor desempenho em relação a Regua Controle (517), será utilizado um teste de duas proporções a fim de manter as proporções de disparos e interesse. 

H0 - A taxa de interesse da Regua 536 é menor que a regua 537

H1 - A taxa de interesse da Regua 536 é maior que a regua 537

In [91]:
pa = sum(base_536['Envios_interacao']) / sum(base_536['Envios_regua'])  # grupo desafiante (536)
pb = sum(base_537['Envios_interacao']) / sum(base_537['Envios_regua'])  # grupo controle (537)
pc = (sum(base_536['Envios_interacao']) + sum(base_537['Envios_interacao'])) / (sum(base_536['Envios_regua']) + sum(base_537['Envios_regua']))

SE = np.sqrt((pc * (1-pc)) * ((1 / sum(base_536['Envios_regua'])) + (1 / sum(base_537['Envios_regua']))))
Z = (pa - pb) / SE

p_valor = (norm.sf(Z))

if p_valor <= 0.05:
    print('Aceitar H1')
else:
    print('Aceitar H0')

Aceitar H0


Segundo o teste de hipóteses não existe evidências estatísticas suficientes para acatar que a régua desafiante (516) desempenhou melhor que a régua controle (517).

***

### Régua 538 x Régua 539

In [92]:
tx_geral_4 = pd.DataFrame({'Regua':['539 (SMS)','538 (Whatsapp)'],
                         'Taxa':[round((sum(base_539["Envios_interacao"]) / sum(base_539["Envios_regua"])) * 100,2),
                                 round((sum(base_538["Envios_interacao"]) / sum(base_538["Envios_regua"])) * 100,2)]})

In [93]:
fig = make_subplots(rows=3,cols=1,shared_yaxes= True, subplot_titles=('Taxa Interesse Régua 539 (SMS)','Taxa Interesse Régua 538 (Whatsapp)','Taxa Geral'))
fig.add_trace(go.Bar(x=base_539["enviado_em"],
                     y=base_539["Tx_Interesse"],
                     marker_color="red",
                     text=base_539["Tx_Interesse"]),row=1,col=1)
fig.add_trace(go.Bar(x=base_538["enviado_em"],
                     y=base_538["Tx_Interesse"],
                     marker_color="blue",
                     text=base_538["Tx_Interesse"]),row=2,col=1)
fig.add_trace(go.Bar(x=tx_geral_4['Regua'],
                     y=tx_geral_4['Taxa'],
                     marker_color="Green",
                     text=tx_geral_4['Taxa']),row=3,col=1)
fig.update_yaxes(range=[0, 100], row=1,col=1)
fig.update_yaxes(range=[0, 100], row=2,col=1)
fig.update_yaxes(range=[0, 100], row=3,col=1)
fig.update_layout(width=1400, height=1250, showlegend= False)

Conforme análise gráfica, a Régua A (SMS) desempenhou melhor que a Régua B, seguirei com alguns testes de hipóteses para validar a questão.

Existe diferença significativa na quantidade disparada nas réguas?

H0: Não existe Diferença


H1: Existe diferença
***

In [94]:
total_envios_4 = pd.DataFrame({'Reguas':['539 (SMS)','538 (Whatsapp)'],
                             'Total Notificações':[base_539['Envios_regua'].sum(),base_538['Envios_regua'].sum()]})

fig = go.Figure()
fig.add_trace(go.Bar(x=total_envios_4['Reguas'],
                     y=total_envios_4['Total Notificações'],
                     marker_color="red",
                     text=total_envios_4['Total Notificações']))
fig.update_layout(title='Total Disparos')

Conforme validações da análise gráfica foi possível ver que houve mais envios na régua referente ao SMS, e isso pode ter impactado o resultado final. Por isso será para o teste de hipótese da Regua Desafiante (523) ter tido um melhor desempenho em relação a Regua Controle (524), será utilizado um teste de duas proporções a fim de manter as proporções de disparos e interesse. 

H0 - A taxa de interesse da Regua 523 é menor que a regua 524

H1 - A taxa de interesse da Regua 523 é maior que a regua 524

In [95]:
pa = sum(base_538['Envios_interacao']) / sum(base_538['Envios_regua'])  # grupo desafiante (538)
pb = sum(base_539['Envios_interacao']) / sum(base_539['Envios_regua'])  # grupo controle (539)
pc = (sum(base_538['Envios_interacao']) + sum(base_539['Envios_interacao'])) / (sum(base_538['Envios_regua']) + sum(base_539['Envios_regua']))

SE = np.sqrt((pc * (1-pc)) * ((1 / sum(base_538['Envios_regua'])) + (1 / sum(base_539['Envios_regua']))))
Z = (pa - pb) / SE

p_valor = (norm.sf(Z))

if p_valor <= 0.05:
    print('Aceitar H1')
else:
    print('Aceitar H0')

Aceitar H0


Segundo o teste de hipóteses não existe evidências estatísticas suficientes para acatar que a régua desafiante (525) desempenhou melhor que a régua controle (526).

***